In [1]:
import pandas as pd
import numpy as np

Lets read all our data into their own dataframes. <br>
Rather than parsing the excel file via the sheets, we can condense our data into a single dataframe/csv

In [216]:
invert_df = pd.read_excel('standard_cell_data_4_cells_V1.xlsx', engine='openpyxl', sheet_name='Inv_1x')
buffer_df = pd.read_excel('standard_cell_data_4_cells_V1.xlsx', engine='openpyxl', sheet_name='buffer_1x')
nand_df = pd.read_excel('standard_cell_data_4_cells_V1.xlsx', engine='openpyxl', sheet_name='nand_1x')
nor_df = pd.read_excel('standard_cell_data_4_cells_V1.xlsx', engine='openpyxl', sheet_name='nor_1x')
invert_df.shape, buffer_df.shape, nand_df.shape, nor_df.shape

((7560, 6), (7560, 6), (7560, 6), (7560, 6))

Need to drop some rows for "FAILED" and negative time values'

In [217]:
buffer_df.drop(buffer_df.index[532:644], axis=0, inplace=True)
buffer_df.shape

(7448, 6)

In [226]:
#standardize column names:
invert_df = invert_df.rename(columns={'clock cycle': 'clock_cycle'})
buffer_df = buffer_df.rename(columns={'clock delay': 'clock_cycle'})
nand_df = nand_df.rename(columns={'Metric': 'Metrics', 'clcok delay': 'clock_cycle'})
nor_df = nor_df.rename(columns={'Metric': 'Metrics', 'clock delay': 'clock_cycle'});

In [227]:
metrics = ['T_RISE ', 'T_FALL ', 'T_DELAY ']
dfs = [invert_df, buffer_df, nand_df, nor_df]

Lets take a look at one:

In [219]:
invert_df.head(4)

,ncfet_inverter,Metrics,clock cycle,thickness tFE,Vdd,time
0,./100ps/0nm/ncfet_inverter_0.3,.,100ps,0nm,0.3,NaN
1,T_RISE,T_RISE,NaN,NaN,NaN,2.410636e-12
2,T_FALL,T_FALL,NaN,NaN,NaN,2.278608e-12
3,T_DELAY,T_DELAY,NaN,NaN,NaN,4.401267e-12


Looks like every 4 rows represents one single reading -- we can create new columns (for 'T_RISE', etc..) that represent rows 1-3 for the same reading/simulation -- lets see/confirm which metric values might be missing or not what we might be expecting

In [221]:
def check_metrics(df):
    return [(df[df.Metrics == metric].time <= 0).sum() for metric in metrics]

In [222]:
check_metrics(dfs[0]), check_metrics(dfs[1]), check_metrics(dfs[2]), check_metrics(dfs[3])

([0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 571])

Rather than trying to remove all the negative values and their associated rows (the other time values for that condition), lets put all of them into their own columns first and then we can drop the entire rows

In [99]:
dfs[0]

,ncfet_inverter,Metrics,clock cycle,thickness tFE,Vdd,time
0,./100ps/0nm/ncfet_inverter_0.3,.,100ps,0nm,0.3,NaN
1,T_RISE,T_RISE,NaN,NaN,NaN,2.410636e-12
2,T_FALL,T_FALL,NaN,NaN,NaN,2.278608e-12
3,T_DELAY,T_DELAY,NaN,NaN,NaN,4.401267e-12
4,./100ps/0nm/ncfet_inverter_0.4,.,100ps,0nm,0.4,NaN
...,...,...,...,...,...,...
7555,T_DELAY,T_DELAY,NaN,NaN,NaN,3.798987e-12
7556,./900ps/9nm/ncfet_inverter_0.9,.,900ps,9nm,0.9,NaN
7557,T_RISE,T_RISE,NaN,NaN,NaN,2.305576e-11
7558,T_FALL,T_FALL,NaN,NaN,NaN,2.296567e-11


In [103]:
dfs[0].iloc[np.arange(0, len(dfs[0]), 4)]

,ncfet_inverter,Metrics,clock cycle,thickness tFE,Vdd,time
0,./100ps/0nm/ncfet_inverter_0.3,.,100ps,0nm,0.3,NaN
4,./100ps/0nm/ncfet_inverter_0.4,.,100ps,0nm,0.4,NaN
8,./100ps/0nm/ncfet_inverter_0.5,.,100ps,0nm,0.5,NaN
12,./100ps/0nm/ncfet_inverter_0.6,.,100ps,0nm,0.6,NaN
16,./100ps/0nm/ncfet_inverter_0.7,.,100ps,0nm,0.7,NaN
...,...,...,...,...,...,...
7540,./900ps/9nm/ncfet_inverter_0.5,.,900ps,9nm,0.5,NaN
7544,./900ps/9nm/ncfet_inverter_0.6,.,900ps,9nm,0.6,NaN
7548,./900ps/9nm/ncfet_inverter_0.7,.,900ps,9nm,0.7,NaN
7552,./900ps/9nm/ncfet_inverter_0.8,.,900ps,9nm,0.8,NaN


In [114]:
def get_base_df(df):
    base_idxs = np.arange(0, len(df), 4)
    base_df = df.iloc[base_idxs]
    return base_df.reset_index(drop=True)

In [116]:
get_base_df(dfs[0])[:4]

,ncfet_inverter,Metrics,clock cycle,thickness tFE,Vdd,time
0,./100ps/0nm/ncfet_inverter_0.3,.,100ps,0nm,0.3,NaN
1,./100ps/0nm/ncfet_inverter_0.4,.,100ps,0nm,0.4,NaN
2,./100ps/0nm/ncfet_inverter_0.5,.,100ps,0nm,0.5,NaN
3,./100ps/0nm/ncfet_inverter_0.6,.,100ps,0nm,0.6,NaN


In [209]:
def get_metrics_df(df):
    base_df = get_base_df(df)
    for metric in metrics:
        base_df[f'{metric}'] = df[df.Metrics == metric]['time'].map(float).values
    #base_df['type'] = base_df.columns[0]
    base_df.insert(loc=1, column='type', value=base_df.columns[0])
    return base_df

In [208]:
dfs[0][dfs[0].Metrics == 'T_RISE '].time.map(float).values

array([2.410636e-12, 2.254149e-12, 1.891338e-12, ..., 1.634674e-11,
       1.971554e-11, 2.305576e-11])

In [210]:
get_metrics_df(dfs[0]).head(2)

,ncfet_inverter,type,Metrics,clock cycle,thickness tFE,Vdd,time,T_RISE,T_FALL,T_DELAY
0,./100ps/0nm/ncfet_inverter_0.3,ncfet_inverter,.,100ps,0nm,0.3,NaN,2.410636e-12,2.278608e-12,4.401267e-12
1,./100ps/0nm/ncfet_inverter_0.4,ncfet_inverter,.,100ps,0nm,0.4,NaN,2.254149e-12,2.258426e-12,2.525869e-12


In [211]:
def clean_df(df):
    first_col = df.columns[0] #if type was inserted at front would need to index second col
    df = df.drop([first_col, 'Metrics', 'time'], axis=1)
    return df

In [213]:
clean_df(get_metrics_df(dfs[0])).head(4)

,type,clock cycle,thickness tFE,Vdd,T_RISE,T_FALL,T_DELAY
0,ncfet_inverter,100ps,0nm,0.3,2.410636e-12,2.278608e-12,4.401267e-12
1,ncfet_inverter,100ps,0nm,0.4,2.254149e-12,2.258426e-12,2.525869e-12
2,ncfet_inverter,100ps,0nm,0.5,1.891338e-12,1.891455e-12,1.588824e-12
3,ncfet_inverter,100ps,0nm,0.6,1.771861e-12,1.770314e-12,1.168491e-12


In [214]:
clean_df(get_metrics_df(dfs[1]))

,type,clock_cycle,thickness tFE,Vdd,T_RISE,T_FALL,T_DELAY
0,ncfet_buffer,100ps,0nm,0.3,3.045711e-12,3.074170e-12,9.658936e-12
1,ncfet_buffer,100ps,0nm,0.4,1.391805e-12,1.444672e-12,5.845318e-12
2,ncfet_buffer,100ps,0nm,0.5,1.261770e-12,1.207603e-12,4.342375e-12
3,ncfet_buffer,100ps,0nm,0.6,1.121240e-12,1.108457e-12,3.483294e-12
4,ncfet_buffer,100ps,0nm,0.7,1.109113e-12,1.107545e-12,3.016897e-12
...,...,...,...,...,...,...,...
1857,ncfet_buffer,900ps,9nm,0.5,3.200329e-12,3.077059e-12,1.109629e-11
1858,ncfet_buffer,900ps,9nm,0.6,3.085752e-12,2.982832e-12,8.822629e-12
1859,ncfet_buffer,900ps,9nm,0.7,3.558769e-12,3.439120e-12,7.870057e-12
1860,ncfet_buffer,900ps,9nm,0.8,4.286678e-12,4.176782e-12,7.407664e-12
